# [教學目標]
- 綜合到目前為止所教的內容, 提交一次作答結果吧!!

# [範例重點]
- 資料清理 (In[4], Out[4]) 
- 前處理 : MinMaxScaler, Imputer (In[4], Out[4])   
- Heatmap 的進階用法 : 散佈圖, KDE, 密度圖 (In[6], Out[6])
- 輸出值的紀錄 (In[8], Out[8])

In [2]:
# Import 需要的套件
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 仿造之前做過的處理

In [38]:
# 設定 data_path
dir_data = './data/Day_021/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

# 讀取檔案
app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

In [39]:
# 種類 2 種以下的類別型欄位轉標籤編碼 (Label Encoding)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # 紀錄有多少個 columns 被標籤編碼過
            le_count += 1
            
# 標籤編碼 (2種類別) 欄位轉 One Hot Encoding            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [40]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

### 做好前處理
開始擬合模型之前，我們要確保 training & testing data 的欄位數量一致，原因是因為 One hot encoding 會製造多的欄位，有些類別出現在 training data 而沒有出現 testing data 中，我們就要把這些多餘的欄位去除

In [41]:
train_labels = app_train['TARGET']

# 調整欄位數, 移除出現在 training data 而沒有出現 testing data 中的欄位
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

In [45]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer


# 特徵欄位清單
train = app_train
features = list(train.columns)

# 複製 test 資料
test = app_test.copy()

# 填補器 : 設定缺失值補中位數
#imputer = Imputer(strategy = 'median')
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# 縮放器 : 設定特徵縮放到 0~1 區間
scaler = MinMaxScaler(feature_range = (0, 1))

# 填補器載入個欄中位數
imputer.fit(train)

# 將中位數回填 train, test 資料中的空缺值
train = imputer.transform(train)
test = imputer.transform(app_test)

# 縮放器載入 train 的上下限, 對 train, test 進行縮放轉換
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 240)
Testing data shape:  (48744, 240)


### Fit the model

In [54]:
train_labels

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

In [46]:
from sklearn.linear_model import LogisticRegression

# 設定模型與模型參數
log_reg = LogisticRegression(C = 0.0001)

# 使用 Train 資料訓練模型
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001)

模型 fit 好以後，就可以用來預測 testing data 中的客戶違約遲繳貸款的機率咯! (記得要用 predict_proba 才會輸出機率)

In [53]:
log_reg.predict_proba(test)

array([[0.93481031, 0.06518969],
       [0.88296282, 0.11703718],
       [0.93224388, 0.06775612],
       ...,
       [0.93346296, 0.06653704],
       [0.92747078, 0.07252922],
       [0.91037404, 0.08962596]])

In [48]:
# 用模型預測結果
# 請注意羅吉斯迴歸是分類預測 (會輸出 0 的機率, 與 1 的機率), 而我們只需要留下 1 的機率這排
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [51]:
log_reg_pred.shape

(48744,)

### 儲存預測結果

In [52]:
# 計算提交結果
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

/tmp/ipykernel_7305/1288466153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['TARGET'] = log_reg_pred


,SK_ID_CURR,TARGET
0,100001,0.065190
1,100005,0.117037
2,100013,0.067756
3,100028,0.071344
4,100038,0.118386


## 練習時間
將你的結果存成 csv, 上傳你的第一份 Kaggle 成績

Hints: https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file

### 一、標準化(Standardization)，或者去除均值和方差進行縮放

公式：(X-X_mean)/X_std 計算時對每個屬性/每列分別進行, Z-score 標準化。

將資料按其屬性(按列進行)減去其均值，然後除以其方差。最後得到的結果是，對每個屬性/每列來說所有資料都聚集在0附近，方差值為1。

首先說明下sklearn中preprocessing庫裡面的scale函式使用方法，預設引數如下：

scale(X, axis=0, with_mean=True, with_std=True, copy=True)

- X：進行標準化的資料，為陣列或矩陣
- axis:當 axis=0 時，所有樣本資料的每個特徵（每列）服從標準正太分佈，均值為0，方差為1,預設為0；當axis=1 時，每個樣本的所有特徵（每行）服從標準正太分佈，均值為0，方差為1
- with_mean:布林型別，預設為True
- with_std：布林型別，預設為True

In [8]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],[ 2.,  0.,  0.],[ 0.,  1., -1.]])
X_scaled=preprocessing.scale(X_train)
X_scaled


array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [9]:
print(X_scaled.mean(axis=0))  ### 計算X_scaled特徵(每列)的均值進行驗證
print(X_scaled.std(axis=0))   ### 計算X_scaled特徵(每列)的方差進行驗證

[0. 0. 0.]
[1. 1. 1.]


In [10]:
(X_train-X_train.mean(axis=0))/X_train.std(axis=0)  ###用原始公式將X_train標準化，結果如下：


array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

## Standardization 平均&變異數標準化

- 將所有特徵標準化，也就是高斯分佈。使得數據的平均值為0，方差為1。
- 適合的使用時機於當有些特徵的方差過大時，使用標準化能夠有效地讓模型快速收斂。

In [11]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(X_train)
scaler

StandardScaler()

In [12]:
print(scaler.mean_)

[1.         0.         0.33333333]


In [13]:
print(scaler.scale_)

[0.81649658 0.81649658 1.24721913]


In [14]:
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

### 二、 將特徵值的取值縮放到一個範圍（0,1）  

除了上述方法，還可以通過將資料特徵值縮放到一個最大、最小區間之中【0,1】，可以用preprocessing.MinMaxScaler實現，預設引數如下：

`MinMaxScaler`(feature_range=(0, 1), copy=True）### 若取值範圍在[-1,1]，則令feature_range=(-1,1)

公式：feature_range=(min, max)
```
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min)   min
```

使用這種方法的目的包括：

- 1、對於方差非常小的屬性可以增強其穩定性；
- 2、維持稀疏矩陣中為0的條目。


In [20]:
from sklearn.preprocessing import MinMaxScaler
print(X_train)
min_max_scaler=MinMaxScaler()
X_train_minmax=min_max_scaler.fit_transform(X_train)
X_train_minmax   

[[ 1. -1.  2.]
 [ 2.  0.  0.]
 [ 0.  1. -1.]]


array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [21]:
### 訓練資料的特徵標準化之後，可以通過transform實現對測試資料或新的資料標準化按照原來的標準化規則
X_test=np.array([[-3,-1,4]])    
X_test_minmax=min_max_scaler.transform(X_test)
X_test_minmax

array([[-1.5       ,  0.        ,  1.66666667]])

### 三、Normalization(L1、L2)  (歸一化)

該過程是將每個樣本縮放到單位範數(每個樣本的範數為1)，如果使用如二次型(點積)或者其它核方法計算兩個樣本之間的相似性這個方法會很有用。

該方法是文字分類和聚類分析中經常使用的向量空間模型（Vector Space Model)的基礎.

Normalization主要思想是對每個樣本計算其p-範數，然後對該樣本中每個元素除以該範數，這樣處理的結果是使得每個處理後樣本的p-範數(l1-norm,l2-norm)等於1。

可以用preprocessing.normalize實現，該函式預設引數如下：

`normalize`(X, norm=’l2’, axis=1, copy=True, return_norm=False)

norm=’l2’時，每個樣本的特徵值為原始值除以各個特徵值的模或絕對值和的平方

norm=’l1’時，變換後變換後每個樣本的各維特徵的絕對值和為1。

**方法1：**

In [22]:
from sklearn.preprocessing import normalize
X = [[ 1., -1.,  2.],[ 2.,  0.,  0.],[ 0.,  1., -1.]]
X_normalized=normalize(X)
X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

**方法2：**

In [24]:
from sklearn.preprocessing import Normalizer
X = [[ 1., -1.,  2.],[ 2.,  0.,  0.],[ 0.,  1., -1.]]
normalizer = Normalizer().fit(X) 
normalizer.transform(X)

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

### 四、Binarization（二值化）

- 特徵的二值化主要是為了將資料特徵轉變成boolean變數。
- 在sklearn中，sklearn.preprocessing.Binarizer函式可以實現這一功能，該函式預設引數如下：

- Binarizer`(threshold=0.0, copy=True)  ###預設閾值為0，即特徵值大於0為1，小於0為0.


In [25]:
from sklearn.preprocessing import Binarizer
X = [[ 1., -1.,  2.],[ 2.,  0.,  0.],[ 0.,  1., -1.]]  
Binarization=Binarizer().fit(X)
Binarization

Binarizer()

In [26]:
Binarization.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

### 五、缺失值處理

由於不同的原因，許多現實中的資料集都包含有缺失值，要麼是空白的，要麼使用NaNs或者其它的符號替代。這些資料無法直接使用scikit-learn分類器直接訓練，所以需要進行處理。幸運地是，sklearn中的Imputer類提供了一些基本的方法來處理缺失值，如使用均值、中位值或者缺失值所在列中頻繁出現的值來替換。sklearn.preprocessing.Imputer可以實現，該函式預設引數如下：

`Imputer`(missing_values=’NaN’, strategy=’mean’, axis=0, verbose=0,copy=True)



In [30]:
import numpy as np  
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

x=np.array([[1, 2], [np.nan, 3], [7, 6]])  
imp.fit(x)

SimpleImputer()

In [31]:
imp.transform(x)  

array([[1., 2.],
       [4., 3.],
       [7., 6.]])

In [32]:
X1 = [[np.nan, 2], [6, np.nan], [7, 6]]  
imp.transform(X1)       ### 按照X資料每列的平均值填充X1中的空值，比如果X是訓練資料，X1是測試資料，X1每列空值則是X每列平均值

array([[4.        , 2.        ],
       [6.        , 3.66666667],
       [7.        , 6.        ]])